In [1]:
#Required Libraries
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Loading the data
dengue_data = pd.read_csv('../../_data/data_dengue_peru.csv', low_memory=False)



C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_41280\61817505.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data_dengue = pd.read_csv('../../_data/data_dengue_peru.csv')


,Año,Semana,Eventos o daños,Departamento,Distrito,Provincia,Ubigeo,Casos
0,2020,43,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0
1,2021,41,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
2,2021,42,Dengue,MOQUEGUA,MOQUEGUA,MARISCAL NIETO,180101,NaN
3,2020,1,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,0.0
4,2020,53,Dengue,CUSCO,MEGANTONI,LA CONVENCION,80914,2.0
...,...,...,...,...,...,...,...,...
172139,2021,49,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172140,2021,49,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN
172141,2021,50,Dengue,MADRE DE DIOS,MADRE DE DIOS,MANU,170203,NaN
172142,2021,50,Dengue,MADRE DE DIOS,TAHUAMANU,TAHUAMANU,170303,NaN


In [4]:
# Correcting Ubigeo and generating ubigeo codes for Departments and Provinces
dengue_data['Ubigeo'] = dengue_data['Ubigeo'].astype(str).str.zfill(6)

dengue_data['ubigeo_departamento'] = dengue_data['Ubigeo'].astype(str).str[:2]
dengue_data['ubigeo_provincia'] = dengue_data['Ubigeo'].astype(str).str[:4]


dengue_data.head(10)

   Ubigeo Ubigeo_Departamento Ubigeo_Provincia
0   80914                  80             8091
1  180101                  18             1801
2  180101                  18             1801
3   80914                  80             8091
4   80914                  80             8091


In [ ]:
dengue_data.dtypes

In [ ]:
# Loading the shapefile for districts
district_shp = gpd.read_file('../../_data/LIMITE_DISTRITAL_2020_INEI/INEI_LIMITE_DISTRITAL.shp')
district_shp.crs

In [ ]:
#Adapting ubigeo
district_shp['UBIGEO']= district_shp['UBIGEO'].astype(str).astype(float)

In [ ]:
#First figure

#  Filtering the data for 2021 cases
#dengue_2021 = dengue_data[dengue_data['Año'] == 2021]

# Merge dengue data with shapefile data
#dengue_district = district_shp.merge(dengue_2021, left_on='UBIGEO', right_on='Ubigeo', how='inner')


In [ ]:
#Third figure

# Group data by department and year
dengue_data_dep = dengue_data.groupby(['ubigeo_departamento', 'Departamento', 'Año',])['Casos'].sum().reset_index()
dengue_data_dep

In [ ]:
# Grouping the shp data by department and year
dept_shp = district_shp.dissolve( by = 'CCDD', as_index=False )
dept_shp = dept_shp[['CCDD', 'geometry','NOMBDEP']]

#Now, we want every region to be associated to a year, so we create complete dataframes with the combinations. This allows us to show the missing values every year

# Creating a DataFrame with unique years and departments (DPTO)
years = dengue_data_dep['Año'].unique()  # Unique years from dengue data
departments = dept_shp[['CCDD']]  # Unique departments from the shapefile

# Creating a DataFrame with the Cartesian product of departments and years
all_combinations = pd.MultiIndex.from_product([departments['CCDD'], years], names=['CCDD', 'Año']).to_frame(index=False)

# Merging with the dengue data
complete_dengue_dept = all_combinations.merge(dengue_data_dep, left_on=['CCDD', 'Año'], right_on=['ubigeo_departamento', 'Año'], how='left')

# Merging the geometry of the departments from the shapefile
dengue_dept_full = dept_shp[['CCDD', 'geometry']].merge(complete_dengue_dept, on='CCDD', how='right')

dengue_dept_full

In [ ]:
# Grouping the shp data by department and year
dept_shp = district_shp.dissolve( by = 'CCDD', as_index=False )
dept_shp = dept_shp[['CCDD', 'geometry','NOMBDEP']]

#Now, we want every region to be associated to a year, so we create complete dataframes with the combinations. This allows us to show the missing values every year

# Creating a DataFrame with unique years and departments (DPTO)
years = dengue_data_dep['Año'].unique()  # Unique years from dengue data
departments = dept_shp[['CCDD']]  # Unique departments from the shapefile

# Creating a DataFrame with the Cartesian product of departments and years
all_combinations = pd.MultiIndex.from_product([departments['CCDD'], years], names=['CCDD', 'Año']).to_frame(index=False)

# Merging with the dengue data
complete_dengue_dept = all_combinations.merge(dengue_data_dep, left_on=['CCDD', 'Año'], right_on=['ubigeo_departamento', 'Año'], how='left')

# Merging the geometry of the departments from the shapefile
dengue_dept_full = dept_shp[['CCDD', 'geometry']].merge(complete_dengue_dept, on='CCDD', how='right')

dengue_dept_full